# Dataset - All

Merge datasets "global", "US", and "Brazil".

In [1]:
import pandas as pd

## Initialize

In [2]:
srcGlobal = "./time_series_covid19_confirmed_global_transformed.csv"
srcUS = "./time_series_covid19_confirmed_US_transformed.csv"
srcBrazil = "../vinicius_da_silva/output_brazil.csv"
srcArgentina = "../vinicius_da_silva/output_argentina.csv"
dest = "./time_series_covid19_confirmed_all.csv"

# Read data
globalDf = pd.read_csv(srcGlobal)
usDf = pd.read_csv(srcUS)
brazilDf = pd.read_csv(srcBrazil)
argentinaDf = pd.read_csv(srcArgentina)

## Data Manipulation

In [3]:
# Drop Brazil from brazilDf (we've subsampled Brazil to regions)
brazilDf = brazilDf[brazilDf.Province_State != "Brazil"]

# Drop Brazil and Argentina data from globalDf
globalDf = globalDf[globalDf.Country_Region != "Brazil"]
# globalDf = globalDf[globalDf.Country_Region != "Argentina"]

# Drop US data from globalDf
globalDf = globalDf[globalDf.Country_Region != "US"]

# Merge datasets
finalDf = pd.concat([globalDf, usDf], axis = 0)
finalDf = pd.concat([finalDf, brazilDf], axis = 0).reset_index(drop = True)
# finalDf = pd.concat([finalDf, argentinaDf], axis = 0).reset_index(drop = True)

In [4]:
finalDf

,Province_State,Country_Region,Lat,Long,Date,Confirmed Cases
0,Afghanistan,Afghanistan,33.93911,67.709953,1/22/20,NaN
1,Albania,Albania,41.15330,20.168300,1/22/20,NaN
2,Algeria,Algeria,28.03390,1.659600,1/22/20,NaN
3,Andorra,Andorra,42.50630,1.521800,1/22/20,NaN
4,Angola,Angola,-11.20270,17.873900,1/22/20,NaN
...,...,...,...,...,...,...
99090,Rio Grande do Sul,Brazil,-30.03300,-51.230000,11/09/20,758.0
99091,Santa Catarina,Brazil,-27.61200,-48.485000,11/09/20,1034.0
99092,Sergipe,Brazil,-10.91700,-37.050000,11/09/20,100.0
99093,Sao Paulo,Brazil,-23.55000,-46.633000,11/09/20,0.0


## Save Dataframe

In [5]:
finalDf.to_csv(dest, index = False)